In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import cmudict
from nltk.corpus import stopwords
import string
from collections import defaultdict
nltk.download('punkt')
nltk.download('cmudict')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rajdipingale/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package cmudict to
[nltk_data]     /Users/rajdipingale/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


True

In [2]:
import re
import gensim
from nltk import sent_tokenize
from gensim.utils import simple_preprocess
from tqdm import tqdm
import numpy as np

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor

In [4]:
def extract_article(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Assuming the article title is within the <title> tag
    title = soup.title.string if soup.title else 'No Title'

    # Extract the article text
    # Here we assume that the main article content is within <article> tag
    # This might need to be adjusted depending on the website's structure
    article_text = ''
    article = soup.find('article')
    if article:
        paragraphs = article.find_all('p')
        for paragraph in paragraphs:
            article_text += paragraph.get_text() + '\n'
    else:
        # Fallback: Extract all text from <p> tags
        paragraphs = soup.find_all('p')
        for paragraph in paragraphs:
            article_text += paragraph.get_text() + '\n'
    
    return title, article_text

In [5]:
# Read the Excel file
df = pd.read_excel('/Users/rajdipingale/Desktop/Projects/Sentiment Analysis and Prediction from Web Articles/Input.xlsx')


In [23]:
# Iterate over each row in the DataFrame
df['Extracted_Article'] = None  # Initialize the column with None or empty strings

# Loop through each row in the DataFrame
for index, row in df.iterrows():
    url_id = row['URL_ID']
    url = row['URL']
    
    try:
        # Extract the title and article text from the URL
        title, article_text = extract_article(url)
        
        # Combine title and article text
        full_article = title + '\n\n' + article_text
        
        # Save the combined text in the new DataFrame column
        df.at[index, 'Extracted_Article'] = full_article
        
        print(f'Successfully extracted and saved article {url_id}')
    except Exception as e:
        print(f'Failed to extract article {url_id} from {url}: {e}')



Successfully extracted and saved article bctech2011
Successfully extracted and saved article bctech2012
Successfully extracted and saved article bctech2013
Successfully extracted and saved article bctech2014
Successfully extracted and saved article bctech2015
Successfully extracted and saved article bctech2016
Successfully extracted and saved article bctech2017
Successfully extracted and saved article bctech2018
Successfully extracted and saved article bctech2019
Successfully extracted and saved article bctech2020
Successfully extracted and saved article bctech2021
Successfully extracted and saved article bctech2022
Successfully extracted and saved article bctech2023
Successfully extracted and saved article bctech2024
Successfully extracted and saved article bctech2025
Successfully extracted and saved article bctech2026
Successfully extracted and saved article bctech2027
Successfully extracted and saved article bctech2028
Successfully extracted and saved article bctech2029
Successfully

Sentiment Analysisa. Remove stopwords


In [6]:
# Function to load stopwords from text files
def load_stopwords(stopwords_dir, encoding='utf-8'):
    stopwords = set()
    for filename in os.listdir(stopwords_dir):
        file_path = os.path.join(stopwords_dir, filename)
        try:
            with open(file_path, 'r', encoding=encoding, errors='ignore') as file:
                for line in file:
                    stopwords.add(line.strip().lower())
        except UnicodeDecodeError as e:
            print(f"Error reading {file_path}: {e}")
    return stopwords

In [7]:
# Function to remove stopwords from text
def remove_stopwords(text, stopwords):
    words = text.split()
    cleaned_text = ' '.join([word for word in words if word.lower() not in stopwords])
    return cleaned_text


In [8]:
# Directory containing the stopword text files
stopwords_dir = '/Users/rajdipingale/Desktop/Projects/Sentiment Analysis and Prediction from Web Articles/StopWords'

# Load the stopwords
stopwords = load_stopwords(stopwords_dir, encoding='utf-8')


In [ ]:
# Create a new column to store the cleaned article text
df['Cleaned_Article'] = None

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    text = row['Extracted_Article']
    
    # Remove stopwords from the text
    cleaned_text = remove_stopwords(text, stopwords)
    
    # Save the cleaned text back into the DataFrame
    df.at[index, 'Cleaned_Article'] = cleaned_text


In [10]:
def load_and_filter_words(master_dict_file, stopwords, encoding='utf-8'):
    words = set()
    with open(master_dict_file, 'r', encoding=encoding, errors='ignore') as file:
        for line in file:
            word = line.strip().lower()
            if word and word not in stopwords:
                words.add(word)
    return words


In [11]:
positive_words_file = '/Users/rajdipingale/Desktop/Projects/Sentiment Analysis and Prediction from Web Articles/MasterDictionary/positive-words.txt'
negative_words_file = '/Users/rajdipingale/Desktop/Projects/Sentiment Analysis and Prediction from Web Articles/MasterDictionary/negative-words.txt'


In [12]:
# Load and filter positive and negative words
positive_words = list(load_and_filter_words(positive_words_file, stopwords, encoding='utf-8'))
negative_words = list(load_and_filter_words(negative_words_file, stopwords, encoding='utf-8'))


In [ ]:
print("Positive Words:", positive_words)
print("Negative Words:", negative_words)

In [32]:

# Function to calculate the scores
def calculate_scores(text, positive_words, negative_words):
    tokens = word_tokenize(text.lower())
    positive_score = sum(1 for word in tokens if word in positive_words)
    negative_score = sum(1 for word in tokens if word in negative_words)
    polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
    return polarity_score*10


In [33]:
# Calculate the polarity scores and add them to the DataFrame
df['Polarity Score'] = df['Cleaned_Article'].apply(lambda x: calculate_scores(x, positive_words, negative_words))


In [34]:
df.head()

,URL_ID,URL,Extracted_Article,Cleaned_Article,Polarity Score
0,bctech2011,https://insights.blackcoffer.com/ml-and-ai-bas...,ML and AI-based insurance premium model to pre...,ML AI-based insurance premium model predict pr...,6.190476
1,bctech2012,https://insights.blackcoffer.com/streamlined-i...,Streamlined Integration: Interactive Brokers A...,Streamlined Integration: Interactive Brokers A...,9.999995
2,bctech2013,https://insights.blackcoffer.com/efficient-dat...,Efficient Data Integration and User-Friendly I...,Efficient Data Integration User-Friendly Inter...,9.999997
3,bctech2014,https://insights.blackcoffer.com/effective-man...,Effective Management of Social Media Data Extr...,Effective Management Social Media Data Extract...,9.999995
4,bctech2015,https://insights.blackcoffer.com/streamlined-t...,Streamlined Trading Operations Interface for M...,Streamlined Trading Operations Interface MetaT...,9.999997


In [38]:
story = []
for doc in df['Cleaned_Article']:
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))

In [39]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [40]:
model.build_vocab(story)


In [41]:
model.train(story, total_examples=model.corpus_count, epochs=model.epochs)


(97970, 121965)

In [42]:
len(model.wv.index_to_key)


2220

In [43]:
def document_vector(doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis=0)

In [46]:
document_vector(df['Cleaned_Article'].values[0])


array([-0.20684879,  0.2020967 , -0.00178433,  0.10398842,  0.01382534,
       -0.48018152,  0.11183865,  0.48722053, -0.18665679, -0.14025079,
       -0.1152152 , -0.35958412, -0.17670296,  0.1642435 ,  0.12704533,
       -0.16552097,  0.03457852, -0.28552842,  0.03209473, -0.4690563 ,
        0.16545556,  0.04257831,  0.18595906, -0.15180638, -0.10106605,
        0.11936492, -0.18276818, -0.14157732, -0.23707482,  0.05992527,
        0.26392105, -0.06218736,  0.08610237, -0.16368271, -0.04899084,
        0.1104377 ,  0.00115286, -0.18862389, -0.03566688, -0.44787744,
        0.08343272, -0.22661938, -0.22572745, -0.03399974,  0.10953996,
       -0.15054241, -0.08578535, -0.11812308,  0.05748444,  0.1799875 ,
        0.04710894, -0.39491308, -0.03773792, -0.04355729, -0.06985509,
        0.17842574,  0.12845874,  0.05221111, -0.37001482,  0.15313001,
        0.06398661, -0.01896054,  0.01136607, -0.02682653, -0.1910794 ,
        0.20587204,  0.09074128,  0.23620519, -0.2801882 ,  0.49

In [48]:
X = []
for doc in tqdm(df['Cleaned_Article'].values):
    X.append(document_vector(doc))

python(49262) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
100%|████████████████████████████████████████| 147/147 [00:00<00:00, 460.46it/s]


In [49]:
X = np.array(X)
X[0]


array([-0.20684879,  0.2020967 , -0.00178433,  0.10398842,  0.01382534,
       -0.48018152,  0.11183865,  0.48722053, -0.18665679, -0.14025079,
       -0.1152152 , -0.35958412, -0.17670296,  0.1642435 ,  0.12704533,
       -0.16552097,  0.03457852, -0.28552842,  0.03209473, -0.4690563 ,
        0.16545556,  0.04257831,  0.18595906, -0.15180638, -0.10106605,
        0.11936492, -0.18276818, -0.14157732, -0.23707482,  0.05992527,
        0.26392105, -0.06218736,  0.08610237, -0.16368271, -0.04899084,
        0.1104377 ,  0.00115286, -0.18862389, -0.03566688, -0.44787744,
        0.08343272, -0.22661938, -0.22572745, -0.03399974,  0.10953996,
       -0.15054241, -0.08578535, -0.11812308,  0.05748444,  0.1799875 ,
        0.04710894, -0.39491308, -0.03773792, -0.04355729, -0.06985509,
        0.17842574,  0.12845874,  0.05221111, -0.37001482,  0.15313001,
        0.06398661, -0.01896054,  0.01136607, -0.02682653, -0.1910794 ,
        0.20587204,  0.09074128,  0.23620519, -0.2801882 ,  0.49

In [50]:
y=df['Polarity Score']

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [54]:
def evaluation_metrics(actual, pred):
    MAE = mean_absolute_error(actual, pred)
    MSE = mean_squared_error(actual, pred)
    RMSE = np.sqrt(mean_squared_error(actual, pred))
    SCORE = r2_score(actual, pred)
    return print(f"R2 Score: {SCORE:.4f} \nMean Absolute Error:{MAE:.4f},\nMean Squared Error:{MSE:.4f} \nRoot Mean Squared Error:{RMSE:.4f}")

In [55]:
model_lr = LinearRegression()
prediction_lr = model_lr.fit(X_train, y_train).predict(X_test)
evaluation_metrics(y_test, prediction_lr)

R2 Score: -6.4537 
Mean Absolute Error:8.7386,
Mean Squared Error:118.5955 
Root Mean Squared Error:10.8902


In [56]:
model_rf = RandomForestRegressor()
prediction_rf = model_rf.fit(X_train, y_train).predict(X_test)
evaluation_metrics(y_test, prediction_rf)

R2 Score: 0.0137 
Mean Absolute Error:3.1913,
Mean Squared Error:15.6929 
Root Mean Squared Error:3.9614


In [57]:
model_svm = SVR()
prediction_svm = model_svm.fit(X_train, y_train).predict(X_test)
evaluation_metrics(y_test, prediction_svm)

R2 Score: 0.0621 
Mean Absolute Error:3.0251,
Mean Squared Error:14.9227 
Root Mean Squared Error:3.8630


In [58]:
model_xgb = XGBRegressor()
prediction_xgb = model_xgb.fit(X_train, y_train).predict(X_test)
evaluation_metrics(y_test, prediction_xgb)

R2 Score: -0.2987 
Mean Absolute Error:3.4592,
Mean Squared Error:20.6633 
Root Mean Squared Error:4.5457


In [60]:
results = {
    'Model': ['  Linear Regression', 'Random Forest', 'Support Vector Machines', ' XGBoost'],
    'MSE': [],
    'RMSE': [],
    'MAE': [],
    'R2_SCORE': []
}

for model_name, pred in zip(results['Model'], [prediction_lr, prediction_rf, prediction_svm, prediction_xgb]):
    results['MSE'].append(mean_squared_error(y_test, pred))
    results['RMSE'].append(np.sqrt(mean_squared_error(y_test, pred)))
    results['MAE'].append(mean_absolute_error(y_test, pred))
    results['R2_SCORE'].append(r2_score(y_test, pred))
    
results = pd.DataFrame(results)
results.sort_values(by="MSE").reset_index(drop = True)

,Model,MSE,RMSE,MAE,R2_SCORE
0,Support Vector Machines,14.922659,3.862986,3.025068,0.062113
1,Random Forest,15.692880,3.961424,3.191322,0.013705
2,XGBoost,20.663349,4.545696,3.459182,-0.298689
3,Linear Regression,118.595517,10.890157,8.738648,-6.453712
